# 深度学习

## 数据预处理

In [2]:
# -*- coding: utf-8 -*-
import re
import pandas as pd
import numpy as np
import jieba
import codecs
import jieba.posseg as pseg
import re

def ad_detect(discuss):
	return discuss
def splitWord(query,stopwords):#分词返回字符串，停用词表是自己找的删除了一些程度描述词
	if len(query)==0:
		query='空信息'
	
	# wordList = []
	# for line in query:
	#     line = line.strip()
	#     words = jieba.cut(line)
	#     wordList.append(words)

	wordList = jieba.lcut(query)
	num = 0
	result = ''
	for word in wordList:
		word = word.rstrip()
		word = word.rstrip('"')
		if word not in stopwords:
			if num == 0:
				result = word
				num = 1
			else:
				result = result + ' ' + word
	return result
def preprocess(data):
	stopwords = {}
	for line in codecs.open('./data/stop.txt','r','utf-8'):
		stopwords[line.rstrip()]=1
	data['split_word'] = data['content'].map(lambda x:splitWord(x,stopwords))
	return data
def select_cn(discuss):
	#print(discuss)
	discuss = re.sub("[A-Za-z0-9\!\%\[\]\,\。\?\？\、\。\，\；\’\【\】\·\~\！\@\#\￥\%\……\&\*\（\）\》\《\：\“\”\{\}\<\/\>\.\ \;\；]", "", discuss)
	return discuss
if __name__ == '__main__':
	train_df = pd.read_csv('./data/data_train.csv',header=None,sep='\t',encoding='gbk')
	test_df = pd.read_csv('./data/data_test.csv',header=None,sep='\t',encoding='gbk')
	train_df.columns=['id','class','content','label']
	test_df.columns=['id','class','content']
	train_df['content']=train_df['content'].fillna('空')
	test_df['content']=test_df['content'].fillna('空') 
	train_df['content'] = train_df['content'].apply(select_cn)
	test_df['content'] = test_df['content'].apply(select_cn)
	train_df = preprocess(train_df)
	test_df = preprocess(test_df)
	train_df.to_csv('./data/train.csv',index=None,encoding='gbk')
	test_df.to_csv('./data/test.csv',index=None,encoding='gbk')

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/vy/jjxqkfb15pj9bsyl9n2zxyww0000gn/T/jieba.cache
Loading model cost 0.648 seconds.
Prefix dict has been built succesfully.


## 利用word2vek训练词向量

### 利用wiki训练的词向量

In [3]:
import codecs
from gensim.models.word2vec import LineSentence
import gensim
from gensim.models import word2vec
in_path='./data/seged_wiki.txt'
_size = 300
model=word2vec.Word2Vec(sentences=LineSentence(in_path), size=_size,sg=1,min_count=5,window=10,workers=30)
model.save('./data/wiki'+str(_size)+'_model.m')
print("all done")

all done


### 利用训练集测试集训练的词向量

In [4]:
import codecs
from gensim.models.word2vec import LineSentence


import gensim
from gensim.models import word2vec
def allget():
	dict1={}
	allword=open('./data/all_fin.csv','w+')
	file=['./data/train.csv','./data/test.csv']
	for filename in file:
		num=0
		with open(filename,encoding='gbk') as fn:
			if 'train' in filename:
				for line in fn:
					num+=1
					if num==1:
						pass
					else:
						if line.strip().split(',')[4] in dict1:
							pass
						else:
							dict1[line.strip().split(',')[4]]='1'
							allword.write(line.strip().split(',')[4]+'\n')
							allword.flush()
			if 'test' in filename:
				for line in fn:
					num+=1
					if num==1:
						pass
					else:
						if line.strip().split(',')[3] in dict1:
							pass
						else:
							dict1[line.strip().split(',')[3]]='1'
							allword.write(line.strip().split(',')[3]+'\n')
							allword.flush()
	allword.close()
#allget()
in_path='./data/all_fin.csv'
_size = 50
model=word2vec.Word2Vec(sentences=LineSentence(in_path), size=_size,sg=1,min_count=5,window=10,workers=30)
model.save('./data/all_fin_train'+str(_size)+'_model.m')

# 利用wiki跟训练集测试集一块训练词向量

In [ ]:
import codecs
from gensim.models.word2vec import LineSentence
import gensim
from gensim.models import word2vec
in_path='./data/ronghe/seged_wiki_all.txt'
_size = 250
model=word2vec.Word2Vec(sentences=LineSentence(in_path), size=_size,sg=1,min_count=5,window=10,workers=30)
model.save('./data/wiki_and_train'+str(_size)+'_model.m')
print("all done")

all done


## 深度学习训练模型（训练得到cnn_Bilstm_unions701_300_100_wiki）

In [1]:
# -*- coding: utf-8 -*-
import pandas  as pd
import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
from  keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import  pad_sequences
from keras.layers import Input, Dropout, Dense, BatchNormalization, Activation, concatenate, GlobalMaxPool1D,GRU, Embedding,Bidirectional, Flatten,LSTM, BatchNormalization,Conv1D,MaxPooling1D
from keras.models import Model
from keras.layers import GlobalMaxPooling1D
from keras.layers import *
from keras.layers.convolutional import Convolution1D
from keras import optimizers
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras import backend as K
from keras import regularizers
import re
from keras.layers import Input, Concatenate
import numpy as np
import gensim
from keras.utils import np_utils
class Model1:
	def __init__(self):
		self.models = gensim.models.Word2Vec.load('./data/wiki_and_train300_model.m')
	def getVec(self,msg):
		return self.models[msg]
new_model = Model1()
train = pd.read_csv('./data/train.csv',encoding='gbk')
test = pd.read_csv('./data/test.csv',encoding='gbk')


maxlen = 60 #每句话最多留多少词
embed_size = 300   #词向量维度
train.split_word.fillna('_na_',inplace=True) # inplace=true 在原数据上改
test.split_word.fillna('_na_',inplace=True)
comment_text = np.hstack([train.split_word.values,test.split_word.values]) # 往后连接起来
tok_raw = Tokenizer() # 对文本中的词进行统计计数，生成文档词典，以支持基于词典位序生成文本的向量表示
tok_raw.fit_on_texts(comment_text)  # fit_on_text(texts) 使用一系列文档来生成token词典，texts为list类，每个元素为一个文档
train['Discuss_seq'] = tok_raw.texts_to_sequences(train.split_word.values) #texts_to_sequences(texts) 将多个文档转换为word下标的向量形式,shape为[len(texts)，len(text)] -- (文档数，每条文档的长度)
word_index = tok_raw.word_index #word_index 一个dict，保存所有word对应的编号id，从1开始
test['Discuss_seq'] = tok_raw.texts_to_sequences(test.split_word.values)




def get_keras_data(dataset): 
	X={
		'Discuss_seq_pre':pad_sequences(dataset.Discuss_seq,maxlen=maxlen)
        # 将dataset.Discuss_seq 规范化成长度为maxlen，长度不够的补0，长度过长的截断
        # keras.preprocessing.sequence.pad_sequences(sequences, maxlen=None, dtype=’int32’, padding=’pre’, truncating=’pre’, value=0.) 
        # 将长为nb_samples的序列（标量序列）转化为形如(nb_samples,nb_timesteps)2D numpy array。
        #如果提供了参数maxlen，nb_timesteps=maxlen，否则其值为最长序列的长度。
        
	}
	return X

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def fbeta_score(y_true, y_pred, beta=1):
    if beta < 0:
        raise ValueError('The lowest choosable beta is zero (only precision).')

    # If there are no true positives, fix the F score at 0 like sklearn.
    if K.sum(K.round(K.clip(y_true, 0, 1))) == 0:
        return 0

    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    bb = beta ** 2
    fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
    return fbeta_score

def fmeasure(y_true, y_pred):
    return fbeta_score(y_true, y_pred, beta=1)


def cnn_lstm():
	#Inputs
	comment_seq_pre = Input(shape=[maxlen],name='Discuss_seq_pre') #用来实例化一个keras张量
	emb_comment1 =Embedding(len(word_index) + 1, embed_size,weights=[embedding_matrix])(comment_seq_pre)
	con = Conv1D(filters=32, kernel_size=3, padding='same', activation='relu')(emb_comment1)
	maxp = MaxPooling1D(pool_size=2)(con)
	lst_1=Dropout(0.5)(LSTM(units=64, activation='tanh')(maxp))
	#lst_1=(LSTM(units=64, activation='tanh',recurrent_dropout=0.2)(maxp))
	output = Dense(3,activation='softmax')(lst_1)
	model = Model(inputs=[comment_seq_pre],outputs=[output])
	adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
	model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["mae", fmeasure])
	return model

from keras.layers import *
from keras.models import *

def cnn_Bilstm():
	def attention_3d_block(inputs):
	    #input_dim = int(inputs.shape[2])
	    a = Permute((2, 1))(inputs)
	    a = Dense(int(maxlen/2), activation='softmax')(a)
	    a_probs = Permute((2, 1), name='attention_vec')(a)
	    #output_attention_mul = merge([inputs, a_probs], name='attention_mul', mode='mul')
	    output_attention_mul = multiply([inputs, a_probs], name='attention_mul')
	    return output_attention_mul

	# build RNN model with attention
	comment_seq_pre = Input(shape=[maxlen],name='Discuss_seq_pre')
	emb_comment1 =Embedding(len(word_index) + 1, embed_size,weights=[embedding_matrix])(comment_seq_pre)
	con = Conv1D(filters=256, kernel_size=3, padding='same', activation='relu')(emb_comment1) #60
	maxp = MaxPooling1D(pool_size=2)(con)
	lstm_out = Bidirectional(LSTM(units=16, return_sequences=True,kernel_regularizer=regularizers.l1(0.01)), name='bilstm')(maxp)
	attention_mul = attention_3d_block(lstm_out)
	attention_flatten = Flatten()(attention_mul)
	drop2 = Dropout(0.5)(attention_flatten)
	#fla = Dense(50)(drop2)
	output = Dense(3,activation='softmax')(drop2)
	model = Model(inputs=[comment_seq_pre],outputs=[output])
	adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
	model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["mae", fmeasure]) # fmeasure 评价指标函数
	return model
def cnn():
	comment_seq_pre = Input(shape=[maxlen],name='Discuss_seq_pre')
	emb_comment1 =Embedding(len(word_index) + 1, embed_size,weights=[embedding_matrix])(comment_seq_pre)
	con1 = Conv1D(filters=32, kernel_size=3, padding='same', activation='relu')(emb_comment1)
	maxp1 = MaxPooling1D(pool_size=2)(con1)

#    con2 = Conv1D(filters=32, kernel_size=3, padding='same', activation='relu')(maxp1)
#    maxp2 = MaxPooling1D(pool_size=2)(con2)
#    con3 = Conv1D(filters=32, kernel_size=3, padding='same', activation='relu')(maxp2)
#    maxp3 = MaxPooling1D(pool_size=2)(con3)

	x = Flatten()(maxp1)  #平滑
	output = Dense(3,activation='softmax')(x)
	model = Model(inputs=[comment_seq_pre],outputs=[output])
	adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
	model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["mae", fmeasure])
	return model

embedding_matrix = np.zeros((len(word_index) + 1, embed_size))
for word, i in word_index.items():
	try:
		embedding_vector = new_model.getVec(word)
		if embedding_vector is not None:
			embedding_matrix[i] = embedding_vector #所有词的词向量
	except:
		pass
		
X_train =get_keras_data(train)
X_test = get_keras_data(test)
y_train = np_utils.to_categorical(train.label.values) #转换成one-hot编码
batch_size = 128#512 #300
epochs = 10
early_stopping = EarlyStopping(monitor="val_loss", mode="min", patience=3)
from keras.models import load_model
callbacks_list = [early_stopping]
#model1 = cnn_lstm()
model1 = cnn_Bilstm()
#model1 = cnn()
model1.summary()
model1.fit(X_train, y_train,
		validation_split=0.1,#zuihou %10作为验证集   十折交叉验证
		batch_size=batch_size, 
		epochs=epochs, 
		shuffle = True,
		callbacks=callbacks_list)

# json_string = model1.to_json()
# open('../model/cnn_lstm.json','w').write(json_string)

# model1.save_weights('../model/cnn_lstm_weights.h5')

      
preds = model1.predict(X_test)
print(preds)
result = []
with open("./result/result_cnn_Bilstm_unions701_300_100_wiki.txt","w") as ipt:
    for line in preds:
        for i in line:
            ipt.write(str(i)+" ")
        ipt.write('\n')
for i in preds:
	r = i.tolist().index(max(i.tolist()))
	result.append(r)
a = range(1,len(result)+1)
a = np.array(a)
df = pd.DataFrame({'ind':a,'pre':result})
df.to_csv('./result/cnn_Bilstm_unions701_300_100_wiki.csv',index = False,header=False)

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


FileNotFoundError: [Errno 2] No such file or directory: './data/wiki_and_train300_model.m.wv.vectors.npy'

## 深度学习训练模型（训练得到result_cnn_Bilstm_616）

In [6]:
# -*- coding: utf-8 -*-
import pandas  as pd
import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
# import fasttext
from  keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import  pad_sequences
from keras.layers import Input, Dropout, Dense, BatchNormalization, Activation, concatenate, GlobalMaxPool1D,GRU, Embedding,Bidirectional, Flatten,LSTM, BatchNormalization,Conv1D,MaxPooling1D
from keras.models import Model
from keras.layers import GlobalMaxPooling1D
from keras.layers import *
from keras.layers.convolutional import Convolution1D
from keras import optimizers
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras import backend as K
from keras import regularizers
import re
from keras.layers import Input, Concatenate
import numpy as np
import gensim
from keras.utils import np_utils
class Model1:
	def __init__(self):
		self.models = gensim.models.Word2Vec.load('./data/wiki_and_train50_model.m')
        #self.models = gensim.models.Word2Vec.load('./data/wiki_and_train300_model.m')
	def getVec(self,msg):
		return self.models[msg]
new_model = Model1()
train = pd.read_csv('./data/train.csv',encoding='gbk')
test = pd.read_csv('./data/test.csv',encoding='gbk')


maxlen = 40
embed_size = 50
train.split_word.fillna('_na_',inplace=True)
test.split_word.fillna('_na_',inplace=True)
comment_text = np.hstack([train.split_word.values,test.split_word.values])
tok_raw = Tokenizer()
tok_raw.fit_on_texts(comment_text)
train['Discuss_seq'] = tok_raw.texts_to_sequences(train.split_word.values)
word_index = tok_raw.word_index
test['Discuss_seq'] = tok_raw.texts_to_sequences(test.split_word.values)




def get_keras_data(dataset): 
	X={
		'Discuss_seq_pre':pad_sequences(dataset.Discuss_seq,maxlen=maxlen)
	}
	return X

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def fbeta_score(y_true, y_pred, beta=1):
    if beta < 0:
        raise ValueError('The lowest choosable beta is zero (only precision).')

    # If there are no true positives, fix the F score at 0 like sklearn.
    if K.sum(K.round(K.clip(y_true, 0, 1))) == 0:
        return 0

    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    bb = beta ** 2
    fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
    return fbeta_score

def fmeasure(y_true, y_pred):
    return fbeta_score(y_true, y_pred, beta=1)


def cnn_lstm():
	#Inputs
	comment_seq_pre = Input(shape=[maxlen],name='Discuss_seq_pre')
	emb_comment1 =Embedding(len(word_index) + 1, embed_size,weights=[embedding_matrix])(comment_seq_pre)
	con = Conv1D(filters=32, kernel_size=3, padding='same', activation='relu')(emb_comment1)
	maxp = MaxPooling1D(pool_size=2)(con)
	lst_1=Dropout(0.5)(LSTM(units=64, activation='tanh')(maxp))
	#lst_1=(LSTM(units=64, activation='tanh',recurrent_dropout=0.2)(maxp))
	output = Dense(3,activation='softmax')(lst_1)
	model = Model(inputs=[comment_seq_pre],outputs=[output])
	adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
	model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["mae", fmeasure])
	return model

from keras.layers import *
from keras.models import *

def cnn_Bilstm():
	def attention_3d_block(inputs):
	    #input_dim = int(inputs.shape[2])
	    a = Permute((2, 1))(inputs)
	    a = Dense(int(maxlen/2), activation='softmax')(a)
	    a_probs = Permute((2, 1), name='attention_vec')(a)
	    #output_attention_mul = merge([inputs, a_probs], name='attention_mul', mode='mul')
	    output_attention_mul = multiply([inputs, a_probs], name='attention_mul')
	    return output_attention_mul

	# build RNN model with attention
	comment_seq_pre = Input(shape=[maxlen],name='Discuss_seq_pre')
	emb_comment1 =Embedding(len(word_index) + 1, embed_size,weights=[embedding_matrix])(comment_seq_pre)
	con = Conv1D(filters=256, kernel_size=3, padding='same', activation='relu')(emb_comment1) #60
	maxp = MaxPooling1D(pool_size=2)(con)
	lstm_out = Bidirectional(LSTM(units=16, return_sequences=True,kernel_regularizer=regularizers.l1(0.01)), name='bilstm')(maxp)
	attention_mul = attention_3d_block(lstm_out)
	attention_flatten = Flatten()(attention_mul)
	drop2 = Dropout(0.5)(attention_flatten)
	#fla = Dense(50)(drop2)
	output = Dense(3,activation='softmax')(drop2)
	model = Model(inputs=[comment_seq_pre],outputs=[output])
	adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
	model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["mae", fmeasure])
	return model
def cnn():
	comment_seq_pre = Input(shape=[maxlen],name='Discuss_seq_pre')
	emb_comment1 =Embedding(len(word_index) + 1, embed_size,weights=[embedding_matrix])(comment_seq_pre)
	con1 = Conv1D(filters=32, kernel_size=3, padding='same', activation='relu')(emb_comment1)
	maxp1 = MaxPooling1D(pool_size=2)(con1)

#    con2 = Conv1D(filters=32, kernel_size=3, padding='same', activation='relu')(maxp1)
#    maxp2 = MaxPooling1D(pool_size=2)(con2)
#    con3 = Conv1D(filters=32, kernel_size=3, padding='same', activation='relu')(maxp2)
#    maxp3 = MaxPooling1D(pool_size=2)(con3)

	x = Flatten()(maxp1)
	output = Dense(3,activation='softmax')(x)
	model = Model(inputs=[comment_seq_pre],outputs=[output])
	adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
	model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["mae", fmeasure])
	return model

embedding_matrix = np.zeros((len(word_index) + 1, embed_size))
for word, i in word_index.items():
	try:
		embedding_vector = new_model.getVec(word)
		if embedding_vector is not None:
			embedding_matrix[i] = embedding_vector
	except:
		pass
		
X_train =get_keras_data(train)
X_test = get_keras_data(test)
y_train = np_utils.to_categorical(train.label.values)
batch_size = 128#512 #300
epochs = 10
early_stopping = EarlyStopping(monitor="val_loss", mode="min", patience=3)
from keras.models import load_model
callbacks_list = [early_stopping]
#model1 = cnn_lstm()
model1 = cnn_Bilstm()
#model1 = cnn()
model1.summary()
model1.fit(X_train, y_train,
		validation_split=0.1,#zuihou %10作为验证集
		batch_size=batch_size, 
		epochs=epochs, 
		shuffle = True,
		callbacks=callbacks_list)

# json_string = model1.to_json()
# open('../model/cnn_lstm.json','w').write(json_string)

# model1.save_weights('../model/cnn_lstm_weights.h5')

      
preds = model1.predict(X_test)
print(preds)
result = []
with open("./result/result_cnn_Bilstm_616.txt","w") as ipt:
    for line in preds:
        for i in line:
            ipt.write(str(i)+" ")
        ipt.write('\n')
for i in preds:
	r = i.tolist().index(max(i.tolist()))
	result.append(r)
a = range(1,len(result)+1)
a = np.array(a)
df = pd.DataFrame({'ind':a,'pre':result})
df.to_csv('./result/cnn_Bilstm_616.csv',index = False,header=False)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Discuss_seq_pre (InputLayer)    (None, 40)           0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 40, 50)       3622150     Discuss_seq_pre[0][0]            
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 40, 256)      38656       embedding_3[0][0]                
__________________________________________________________________________________________________
max_pooling1d_3 (MaxPooling1D)  (None, 20, 256)      0           conv1d_3[0][0]                   
__________________________________________________________________________________________________
bilstm (Bi

In [4]:
a = {'a':[1,2,3],'b':[4,5,6]}
import pandas as pd
import copy
b = pd.DataFrame(a)
b

,a,b
0,1,4
1,2,5
2,3,6


In [5]:
c = copy.deepcopy(b)
c['a'][1] =  9
c

,a,b
0,1,4
1,9,5
2,3,6


In [10]:
b

,a,b
0,1,4
1,2,5
2,3,6


In [11]:
'test'+str(0) = b
'test'+str(0)

SyntaxError: can't assign to operator (<ipython-input-11-3f83be8f96fc>, line 1)